In [1]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time 
    #json library
import json
    #Analyst tools
import sys
sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analysts_tools.redash_methods import *
from analystcommunity.read_connection_data_warehouse import run_read_dwd_query

import random
import time
import datetime

import seaborn as sns
import matplotlib.pyplot as plt


from datetime import datetime, timedelta

In [2]:
# Get the current date
current_date = datetime.now()
# Set the current month to the first day of the month
current_month = current_date.replace(day=1)
# Subtract a month to exclude the current month
current_month -= timedelta(days=1)
# Generate the last 12 months
last_12_months = [(current_month - timedelta(days=30 * i)).strftime("%Y%m") for i in range(1, 14)]

print(last_12_months)

['202401', '202312', '202311', '202310', '202309', '202308', '202307', '202306', '202305', '202304', '202303', '202302', '202301']


In [3]:
def customers_offer(grupo):
    query = """
    SELECT DISTINCT 
        bcs.customer_segment_id, 
        bcs.external_identifier::int AS customer_id
    FROM postgres_broadleaf_federate."broadleaf.blc_customer_customer_seg_xref" bcs
    WHERE bcs.customer_segment_id = {clientes}
    """.format(clientes=grupo)

    df_clients = read_connection_data_warehouse.runQuery(query)
    return df_clients.customer_id.unique()

In [4]:
def get_info_sd(ciudad,mes):
    query = """
    SELECT 
        bs.site_identifier_value AS region_code,
        bo.offer_id,
        MAX(bcs.customer_segment_id) AS customer_segment_id_test,
        MAX(bcs2.customer_segment_id) AS customer_segment_id_control,
        DATE(bo.start_date) AS start_date,
        DATE(bo.end_date) AS end_date,
        replace(
            substring(
            boic.order_item_match_rule, 
            position('["' in boic.order_item_match_rule) + 2, 
            position('"]' in boic.order_item_match_rule) - position('["' in boic.order_item_match_rule) - 2
        ), '"', '') AS values_string
        
    FROM postgres_broadleaf_federate."broadleaf.blc_offer"                          bo
    LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_offer_customer_seg_xref"   bocsx   ON (bo.offer_id = bocsx.offer_id AND ((bocsx.archived is NULL OR bocsx.archived='N')))
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_tar_crit_offer_xref"      btcofx  ON (btcofx.offer_id = bo.offer_id)
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_offer_item_criteria"      boic    ON (btcofx.offer_item_criteria_id = boic.offer_item_criteria_id and (boic.archived is null or boic.archived ='N'))
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                     bs      ON bs.site_id = bo.catalog_disc
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_customer_segment"         bcs     ON bocsx.customer_segment_id = bcs.customer_segment_id AND bcs.customer_segment_name ILIKE '%piloto%'
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_customer_segment"         bcs2    ON REPLACE(bcs.customer_segment_name, 'piloto', 'control') = bcs2.customer_segment_name 

    WHERE offer_name ILIKE '%\_piloto\_NB\_%'
        AND LEFT(offer_name,6) = '{month}'
        AND bs.site_identifier_value = '{city}'
        AND bo.offer_discount_type = 'PERCENT_OFF'
        AND bo.offer_type = 'ORDER_ITEM'
        AND bo.sndbx_id is NULL
        AND (bo.archived is NULL OR bo.archived='N')
        AND (bo.sndbx_tier is NULL OR bo.sndbx_tier = 999999)
        AND bo.automatically_added = 'true'
    GROUP BY 1,2,5,6,7
    """.format(month=mes,city=ciudad)

    return read_connection_data_warehouse.runQuery(query)

In [5]:
a = get_info_sd('BOG','202401')
a

,region_code,offer_id,customer_segment_id_test,customer_segment_id_control,start_date,end_date,values_string
0,BOG,689675,558545,558509,2024-01-01,2024-01-19,"464556,419097,464554,419124,419122,557467"
1,BOG,689254,559128,559054,2024-01-01,2024-01-19,"164993,164991,250349,164989,164995,426647"
2,BOG,689404,559205,559067,2024-01-01,2024-01-19,"271134,643289,573510,257289,257291"
3,BOG,689808,559045,559007,2024-01-01,2024-01-19,"615638,146297,642919,516866,341539,341551,633515"
4,BOG,689431,558326,559093,2024-01-01,2024-01-19,"615363,197545,558521,640305,75462,610013,75345..."
...,...,...,...,...,...,...,...
305,BOG,689354,558963,558794,2024-01-01,2024-01-19,"641589,639011,616643,639007,639013"
306,BOG,689255,559129,559055,2024-01-01,2024-01-19,"47547,338862,-300180,642089,637467"
307,BOG,689813,559402,559014,2024-01-01,2024-01-19,"161739,130690"
308,BOG,689819,559405,559017,2024-01-01,2024-01-19,"61745,621883"


In [6]:
a['customer_ids_test'] = a['customer_segment_id_test'].apply(customers_offer)
a['customer_ids_control'] = a['customer_segment_id_control'].apply(customers_offer)

In [21]:
a.loc[a.offer_id== 689255,'customer_ids_test'].values[0]

array([ 94189744,  34996676,   -236639, ..., 355754673,  16018294,
       117426218])

In [29]:
def get_values_month(df_sd,city,mes):
    info = []
    for i in df_sd.offer_id.unique():
        customers = tuple(df_sd.loc[df_sd.offer_id== i,'customer_ids_test'].values[0]) 
        product = tuple(map(int, df_sd.loc[df_sd.offer_id== i,'values_string'].values[0].split(',')))

        if len(product) == 1:
            product = product + (1,)
            
        query = """
        SELECT 
            s.identifier_value AS region_code,
            dc.source_id AS customer_id,
            cat.description AS subcategoria,
            MIN(DATE_TRUNC('MONTH', fs.order_submitted_date)) AS firstm

        FROM dpr_sales.fact_sales                   fs
        INNER JOIN dpr_shared.dim_customer          dc  ON dc.customer_id = fs.dim_customer
        INNER JOIN dpr_shared.dim_site              s   ON s.site_id = fs.dim_site
        INNER JOIN dpr_shared.dim_product           dp  ON dp.product_id = fs.dim_product
        INNER JOIN dpr_shared.dim_category          cat ON cat.category_id = dp.category_id
        LEFT JOIN dpr_sales.fact_sales_discounts    fsd ON fs.order_item_id = fsd.order_item_id 

        WHERE 
            fs.gmv_enabled = TRUE
            AND fulfillment_order_status NOT IN ('CANCELLED', 'ARCHIVED','No value')
            AND fs.fb_order_status_id  IN (1,6,7,8)
            AND fs.is_deleted = FALSE
            AND fs.dim_status = 1
            AND dp.is_slot = 'false'
            AND cat.active = 1
            AND dc.source_id IN {users}
            AND dp.source_id IN {products}
            AND s.identifier_value = '{ciudad}'
            AND DATE(fs.order_submitted_date) >= DATE_TRUNC('month', date(CURRENT_DATE) - interval '{mes1} month')
            AND DATE(fs.order_submitted_date) < DATE_TRUNC('month', date(CURRENT_DATE) - interval '{mes2} month')
        GROUP BY 1,2,3
        """.format(ciudad=city,users=customers,products=product,mes1=mes,mes2=mes-1)
    
        df_aux = run_read_dwd_query(query)
        
        if len(df_aux) != 0:
            info.append(df_aux)
            
    df_final = pd.concat(info, ignore_index=True)
    
    return df_final

In [30]:
info = []
for city in ['BOG','CMX', 'SPO', 'MDE', 'BHZ', 'BAQ', 'BOG', 'CWB', 'VCP']:
    mes = 1
    for month in last_12_months:
        df = get_info_sd(city,month)
        # Apply the get_customer_ids function to the 'customer_segment_id_test' and 'customer_segment_id_control' columns
        df['customer_ids_test'] = df['customer_segment_id_test'].apply(customers_offer)
        df['customer_ids_control'] = df['customer_segment_id_control'].apply(customers_offer)
        
        if len(df) != 0:
            df2 = get_values_month(df,city,mes)
            info.append(df2)
            print(f'Succes for {city} AND {month}, len: {len(df2)}')
            mes += 1
        else:
            print(f'NOT Succes for {city} AND {month}')
            mes += 1
    print(f'{city} done')    
combined_df = pd.concat(info, ignore_index=True)

Succes for BOG AND 202401, len: 13569


KeyboardInterrupt: 

In [38]:
df_sd.region_code.unique()

array(['SPO', 'MDE', 'BHZ', 'BAQ', 'BOG', 'CWB', 'VCP'], dtype=object)

In [18]:
def customers_offer(grupo):
    query = """
    SELECT DISTINCT 
        bcs.customer_segment_id, 
        bcs.external_identifier::int AS customer_id
    FROM postgres_broadleaf_federate."broadleaf.blc_customer_customer_seg_xref" bcs
    WHERE bcs.customer_segment_id = {clientes}
    """.format(clientes=grupo)

    df_clients = read_connection_data_warehouse.runQuery(query)
    return df_clients.customer_id.unique()

In [19]:
# Apply the get_customer_ids function to the 'customer_segment_id_test' and 'customer_segment_id_control' columns
df_sd['customer_ids_test'] = df_sd['customer_segment_id_test'].apply(customers_offer)
df_sd['customer_ids_control'] = df_sd['customer_segment_id_control'].apply(customers_offer)

# Drop the original columns if needed
df_sd = df_sd.drop(['customer_segment_id_test', 'customer_segment_id_control'], axis=1)

In [34]:
query = """
SELECT 
    s.identifier_value AS region_code,
    dc.source_id AS customer_id,
    cat.description AS subcategoria,
    MIN(DATE_TRUNC('MONTH', fs.order_submitted_date)) AS firstm

FROM dpr_sales.fact_sales                   fs
INNER JOIN dpr_shared.dim_customer          dc  ON dc.customer_id = fs.dim_customer
INNER JOIN dpr_shared.dim_site              s   ON s.site_id = fs.dim_site
INNER JOIN dpr_shared.dim_product           dp  ON dp.product_id = fs.dim_product
INNER JOIN dpr_shared.dim_category          cat ON cat.category_id = dp.category_id
LEFT JOIN dpr_sales.fact_sales_discounts    fsd ON fs.order_item_id = fsd.order_item_id 

WHERE 
    fs.gmv_enabled = TRUE
    AND fulfillment_order_status NOT IN ('CANCELLED', 'ARCHIVED','No value')
    AND fs.fb_order_status_id  IN (1,6,7,8)
    AND fs.is_deleted = FALSE
    AND fs.dim_status = 1
    AND dp.is_slot = 'false'
    AND cat.active = 1
    AND dc.source_id IN {users}
    AND dp.source_id IN {products}
    --AND s.identifier_value = '{ciudad}'
    AND DATE(fs.order_submitted_date) >= DATE_TRUNC('month', date(CURRENT_DATE) - interval '1 month')
    AND DATE(fs.order_submitted_date) < DATE_TRUNC('month', date(CURRENT_DATE) - interval '0 month')
GROUP BY 1,2,3
""".format(ciudad='SPO',users=tuple(df_sd.customer_ids_test[0]),products=tuple(map(int, df_sd.values_string[0].split(','))))

df_aux = run_read_dwd_query(query)

In [35]:
df_aux

,region_code,customer_id,subcategoria,firstm
0,SPO,115446194,Embutidos,2024-01-01
1,SPO,182680929,Embutidos,2024-01-01
2,SPO,153952745,Embutidos,2024-01-01
3,SPO,199805281,Embutidos,2024-01-01
4,SPO,182833738,Embutidos,2024-01-01
...,...,...,...,...
110,SPO,344151919,Embutidos,2024-01-01
111,SPO,143109792,Embutidos,2024-01-01
112,SPO,179377804,Embutidos,2024-01-01
113,SPO,60769715,Embutidos,2024-01-01


In [15]:
def get_info(city):
    query = """
    WITH 
    base_month AS (
    SELECT 
        s.identifier_value AS region_code,
        dc.source_id AS customer_id,
        cat.description AS subcategoria,
        MIN(DATE_TRUNC('MONTH', fs.order_submitted_date)) AS firstm

    FROM dpr_sales.fact_sales                   fs
    INNER JOIN dpr_shared.dim_customer          dc  ON dc.customer_id = fs.dim_customer
    INNER JOIN dpr_shared.dim_site              s   ON s.site_id = fs.dim_site
    INNER JOIN dpr_shared.dim_product           dp  ON dp.product_id = fs.dim_product
    INNER JOIN dpr_shared.dim_category          cat ON cat.category_id = dp.category_id
    LEFT JOIN dpr_sales.fact_sales_discounts    fsd ON fs.order_item_id = fsd.order_item_id 

    WHERE 
        fs.gmv_enabled = TRUE
        AND fulfillment_order_status NOT IN ('CANCELLED', 'ARCHIVED','No value')
        AND fs.fb_order_status_id  IN (1,6,7,8)
        AND fs.is_deleted = FALSE
        AND fs.dim_status = 1
        AND dp.is_slot = 'false'
        AND cat.active = 1
        AND s.identifier_value = '{ciudad}'
        AND DATE(fs.order_submitted_date) >= DATE_TRUNC('month', date(CURRENT_DATE) - interval '12 month')
        AND DATE(fs.order_submitted_date) < DATE_TRUNC('month', date(CURRENT_DATE) - interval '0 month')
        AND fsd.adjustment_reason ILIKE '%\_NB\_%'
    GROUP BY 1,2,3
    ),

    next_months AS (
    SELECT DISTINCT
        s.identifier_value AS region_code,
        dc.source_id AS customer_id,
        cat.description AS subcategoria,
        DATE_TRUNC('month', fs.order_submitted_date) AS month

    FROM dpr_sales.fact_sales                   fs
    INNER JOIN dpr_shared.dim_customer          dc  ON dc.customer_id = fs.dim_customer
    INNER JOIN dpr_shared.dim_site              s   ON s.site_id = fs.dim_site
    INNER JOIN dpr_shared.dim_product           dp  ON dp.product_id = fs.dim_product
    INNER JOIN dpr_shared.dim_category          cat ON cat.category_id = dp.category_id
    INNER JOIN base_month                       bm  ON dc.source_id = bm.customer_id AND cat.description = bm.subcategoria AND DATE(fs.order_submitted_date) >= firstm AND s.identifier_value = bm.region_code

    WHERE 
        fs.gmv_enabled = TRUE
        AND fulfillment_order_status NOT IN ('CANCELLED', 'ARCHIVED','No value')
        AND fs.fb_order_status_id  IN (1,6,7,8)
        AND fs.is_deleted = FALSE
        AND fs.dim_status = 1
        AND dp.is_slot = 'false'
        AND s.identifier_value = '{ciudad}'
        AND DATE(fs.order_submitted_date) < DATE_TRUNC('month', date(CURRENT_DATE) - interval '0 month')
    ),

    bucket_count AS (
    SELECT 
        region_code,
        firstm,
        subcategoria,
        COUNT(DISTINCT customer_id) AS total_usuarios
    FROM base_month
    GROUP BY 1,2,3
    )

    SELECT --DISTINCT
    bmt.region_code AS city,
    bmt.subcategoria,
    TO_CHAR(bmt.firstm,'YY-MM') AS firstm,
    nmt.month AS month,
    datediff(month,bmt.firstm,nmt.month) AS cohort_number,
    b.total_usuarios,
    COUNT(DISTINCT nmt.customer_id) AS usuarios,
    (usuarios*100.00/b.total_usuarios)::FLOAT AS cohort

    FROM base_month                                                 bmt
    LEFT JOIN next_months                                           nmt     ON nmt.customer_id = bmt.customer_id AND nmt.subcategoria = bmt.subcategoria AND nmt.region_code = bmt.region_code
    LEFT JOIN bucket_count                                          b       ON b.firstm = bmt.firstm AND b.subcategoria = bmt.subcategoria AND b.region_code = bmt.region_code
    GROUP BY 1,2,3,4,5,6
    """.format(ciudad=city)
    
    return run_read_dwd_query(query)

In [13]:
info = []
for city in ('SPO','BOG','BAQ','MDE','CWB','BHZ','VCP','CMX'):
    results = get_info(city)
    info.append(results)
    print(f'Succes for {city}, len: {len(results)}')
combined_df = pd.concat(info, ignore_index=True)

Succes for SPO, len: 3334
Succes for BOG, len: 2801
Succes for BAQ, len: 2667
Succes for MDE, len: 2137
Succes for CWB, len: 2871
Succes for BHZ, len: 2975
Succes for VCP, len: 2993
Succes for CMX, len: 3064


In [14]:
combined_df.to_excel('RETENTION_SD.xlsx')